#**FACE RECOGNITION**

---
Nna

[1] IEEE |  [Face Recognition using Back Propagation Neural
Network Technique](https://ieeexplore.ieee.org/document/7164700) </br>
[2] Machine Learning Mastery - Jason Brownlee | [Weight Initialization for Deep Learning Neural Networks](https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/)

#**ALGORITMA**
---


1.   Ekstrak "*face value*" untuk dataset. </br>
    - Greyscale
    - Resize
    - Flatten + Normalisasi
2.   Pemrosesan dataset dengan BPNN</br>
    - **Vektoral**
    - Matriks



##**Import Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
import random
import string
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import pickle
import gzip
import os
import time
start_time = time.time()
#import matplotlib.pyplot as plt
import hashlib
from collections import defaultdict

##**Data Set**

In [99]:
# Load dataset flatten
dataset = np.load("/content/drive/My Drive//FaceRec_NN/Data_Flatten/Flattened_dataset.npz")
data = dataset['images']
labels = dataset['labels']

print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

Data shape: (1200, 900), Labels shape: (1200, 15)


In [100]:
columns = [f'X_{i}' for i in range(data.shape[1])] + [f'T_{i}' for i in range(labels.shape[1])]
data_with_labels = np.column_stack([data, labels])

df_dataset = pd.DataFrame(data_with_labels, columns=columns)
df_dataset

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,T_5,T_6,T_7,T_8,T_9,T_10,T_11,T_12,T_13,T_14
0,0.462745,0.858824,0.984314,0.894118,0.929412,0.913725,0.847059,0.905882,0.850980,0.878431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.788235,0.807843,0.705882,0.792157,0.807843,0.827451,0.815686,0.835294,0.827451,0.831373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.772549,0.741176,0.741176,0.729412,0.839216,0.815686,0.847059,0.870588,0.874510,0.858824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.741176,0.623529,0.686275,0.588235,0.647059,0.654902,0.501961,0.458824,0.662745,0.745098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.800000,0.745098,0.600000,0.600000,0.552941,0.600000,0.560784,0.356863,0.474510,0.698039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.258824,0.298039,0.262745,0.678431,0.854902,0.862745,0.866667,0.815686,0.835294,0.835294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1196,0.827451,0.835294,0.843137,0.843137,0.839216,0.835294,0.839216,0.843137,0.835294,0.882353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1197,0.823529,0.827451,0.831373,0.835294,0.839216,0.835294,0.831373,0.827451,0.827451,0.819608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1198,0.180392,0.231373,0.270588,0.223529,0.172549,0.211765,0.239216,0.192157,0.168627,0.168627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


##**Implementasi Kode**

**Fungsi Aktivasi**

In [ ]:
# Fungsi: sigmoid
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def d_sigmoid(x):
    return x * (1.0 - x)


# Fungsi: reLU
def relu(x):
    return np.maximum(0, x)

def d_relu(x):
    return np.where(x > 0, 1, 0)


# Fungsi: softmax
def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps)

def d_softmax(x):
    s = softmax(x)
    return s * (1.0 - s)

**Insialisasi**

Spesifikasi

In [ ]:
epoch = 1000

NI = 900             # input neuron (node)
NH = 3               # hidden neuron (node)
hiddenLayer = 1
NO = 15              # output neuron (node)

Bobot (Weight)

In [ ]:
def weightsd(NI, NH, NO):
    """
    Fungsi rentang bobot melalui He Weight Initializaiton
    untuk fungsi aktivasi ReLU

    """
    np.random.seed(42)
    std = math.sqrt(2.0 / NI)                   # standar deviasi bobot

    v_weights = np.random.randn(NI, NH) * std   # bobot input-hidden
    vb_weights = np.random.randn(1, NH) * std   # bobot bias-hidden
    w_wights = np.random.randn(NH, NO) * std    # bobot hidden-output
    wb_weights = np.random.randn(1, NO) * std   # bobot bias-output

    return v_weights, vb_weights, w_wights, wb_weights

In [ ]:
v, vb, w, wb = weightsd(NI, NH, NO)

print("Rentang: ", v.min(), v.max())
print("v =\n", v, "\n", np.shape(v))

print("\nRentang: ", vb.min(), vb.max())
print("vb =\n", vb, "\n", np.shape(vb))

print("\nRentang: ", w.min(), w.max())
print("w =\n", w, "\n", np.shape(w))

print("\nRentang: ", wb.min(), wb.max())
print("wb =\n", wb, "\n", np.shape(wb))

Rentang:  -0.1527948077200883 0.1816195042088606
v =
 [[ 0.02341533 -0.00651784  0.03053233]
 [ 0.07179632 -0.0110381  -0.01103732]
 [ 0.07444481  0.03617722 -0.02213123]
 ...
 [-0.01150606 -0.00185296 -0.00634024]
 [ 0.01573831  0.06747528  0.05099497]
 [-0.06185861  0.02932466  0.06264652]] 
 (900, 3)

Rentang:  0.018234328533338142 0.0948593385157525
vb =
 [[0.01823433 0.05142931 0.09485934]] 
 (1, 3)

Rentang:  -0.11924460562376078 0.09367095911431697
w =
 [[ 0.04825813  0.01175252  0.04926594  0.00683008  0.0011268  -0.01656583
   0.07370001 -0.0385811   0.07223651  0.02356382 -0.06596393  0.01733115
  -0.0989962   0.02948673  0.04173021]
 [-0.02792393  0.00582409  0.09211986 -0.02384113 -0.04991738  0.06984628
   0.09251725  0.00017422  0.04768082  0.06323778 -0.03500104 -0.02287754
   0.05802398  0.07943233  0.02653446]
 [-0.04147485  0.09367096 -0.02503021 -0.01579855  0.01613796  0.07326382
   0.04025681  0.01955695  0.02183963  0.0020513   0.02631095 -0.11924461
  -0.01398221

**BPNN**

In [116]:
class BPNN:
    def __init__(self, NI, NH, NO, lr = 0.5):
        # inisialisasi neuron
        self.ni = NI   # input neuron
        self.nh = NH   # hidden neuron
        self.no = NO   # output neuron
        self.lr = lr   # learning rate

        # inisialisasi parameter feed_forward
        self.z_in = None
        self.z = None
        self.y_in = None
        self.y = None

        # inisialisasi parameter backPropagation_error
        self.delta_y = None
        self.delta_w = None
        self.delta_wb = None
        self.delta_zin = None
        self.delta_z = None
        self.delta_v = None
        self.delta_vb = None


    def feed_forward(self, x, v, vb, w, wb):
        # sinyal unit hidden
        self.z_in = np.dot(x, v) + vb        # sinyal input unit hidden
        self.z = relu(self.z_in)             # sinyal output unit hidden

        # sinyal unit output
        self.y_in = np.dot(self.z, w) + wb  # sinyal input unit output
        self.y = softmax(self.y_in)  # sinyal output unit output

        return self.y, self.z, self.z_in, self.y_in  # Corrected the return order


    def backPropagation_error(self, x, t, z, z_in, y, y_in, v, vb, w, wb):
        # error unit output
        self.delta_y = (t - y) * d_softmax(y_in)  # informasi error

        self.delta_w = np.zeros((self.nh, self.no))  # koreksi bobot unit output
        self.delta_w = self.lr * np.outer(z, self.delta_y)

        self.delta_wb = self.lr * self.delta_y  # koreksi bobot bias unit output

        # error unit hidden
        self.delta_zin = np.zeros((1, self.nh))  # koreksi error
        self.delta_zin = np.dot(self.delta_y, w.T)

        self.delta_z = self.delta_zin * d_relu(z_in)  # aktivasi koreksi error

        self.delta_v = np.zeros((self.ni, self.nh))  # koreksi bobot unit hidden
        self.delta_v = self.lr * np.outer(x, self.delta_z[0])

        self.delta_vb = self.lr * self.delta_z  # koreksi bobot bias unit hidden

        return self.delta_v, self.delta_vb, self.delta_w, self.delta_wb  # Corrected return order


    def update_weights(self, v, vb, w, wb, delta_v, delta_vb, delta_w, delta_wb):
        v += self.delta_v
        vb += self.delta_vb
        w += self.delta_w
        wb += self.delta_wb

        return v, vb, w, wb

**Implementasi training data**

In [119]:
def train(BP, X, T, v, vb, w, wb, epochs=100):
    galat = []  # Untuk menyimpan error atau metrik lainnya

    for e in range(epochs):
        print("Epoch: ", e + 1)

        total_error = 0
        total_error_kuadratic = 0
        total_error_metric = 0

        for i in range(len(X)):
            x = X[i]  # Input data
            t = T[i]  # Target data

            # Feedforward
            y, z, z_in, y_in = BP.feed_forward(x, v, vb, w, wb)

            # Backpropagation
            delta_w, delta_wb, delta_v, delta_vb = BP.backPropagation_error(x, t, z, z_in, y, y_in, v, vb, w, wb)

            # Update bobot dan bias
            v, vb, w, wb = BP.update_weights(v, vb, w, wb, delta_w, delta_wb, delta_v, delta_vb)

            # Menghitung error
            sample_error = -np.sum(t * np.log(y) + (1 - t) * np.log(1 - y))  # Cross-entropy
            total_error += sample_error
            error_kuadratic = 0.5 * np.sum((t - y) ** 2)  # Error kuadratik
            total_error_kuadratic += error_kuadratic
            error_metric = np.sum(np.log(1 - (t - y) ** 2))  # Error metrik
            total_error_metric += error_metric

        # Print hasil per epoch
        average_error = total_error / len(X)

        # Menyimpan data error untuk analisis lebih lanjut
        galat.append([total_error, total_error_kuadratic, total_error_metric])

    # Menampilkan data error setelah training
    df_data = pd.DataFrame(galat, columns=["Total Error", "Total Error Kuadratik", "Total Error Metric"])
    return df_data

##**Hasil**

In [120]:
bpnn = BPNN(NI, NH, NO, lr = 0.5)

# Melatih jaringan
df_data = train(bpnn, data, labels, v, vb, w, wb, epochs=100)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Epoch:  50
Epoch:  51
Epoch:  52
Epoch:  53
Epoch:  54
Epoch:  55
Epoch:  56
Epoch:  57
Epoch:  58
Epoch:  59
Epoch:  60
Epoch:  61
Epoch:  62
Epoch:  63
Epoch:  64
Epoch:  65
Epoch:  66
Epoch:  67
Epoch:  68
Epoch:  69
Epoch:  70
Epoch:  71
Epoch:  72
Epoch:  73
Epoch:  74
Epoch:  75
Epoch:  76
Epoch:  77
Epoch:  78
Epoch:  79
Epoch:  80
Epoch:  81
Epoch:  82
Epoch:  83
Epoch:  84
Epoch:  85
Epoch:  86
Epoch:  87
Epoch:  88
Epoch:  89
Epoch:  90
Epoch:  91
Epoch:  

#**CONCLUSION**
---
